In this notebook I'll run different models on these data.



In [33]:
from algorithm import (
                        CSVDataLoader,
                        )
import matplotlib.pyplot as plt
import numpy as np
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import integrated_brier_score

In [39]:
dl = CSVDataLoader()
X, Y, _, _, _, time_all, event_all = dl.get_data(num_intervals=7)
Y = Y.sum(axis=1).cpu().numpy()
X = X.cpu().numpy()
dtype = [('cens', bool), ('time', int)]
Y = np.array(list(zip(event_all.astype(bool), Y)), dtype=dtype)



train_indices, test_indices = train_test_split(
        range(len(X)), stratify=event_all, random_state=1, test_size=0.25
    )
X_train, X_test, y_train, y_test = X[train_indices], X[test_indices], Y[train_indices], Y[test_indices]




# Random Survival Forest

In [40]:

rsf = RandomSurvivalForest(max_depth=2, random_state=1)
rsf.fit(X_train, y_train)
print(f"Random Forest C-Index : {rsf.score(X_test, y_test)}")
survs = rsf.predict_survival_function(X_test)
_times = np.arange(1, 7)
preds = np.asarray([[fn(t) for t in _times] for fn in survs])
print(f"Random Forest Integrated Brier Score : {integrated_brier_score(Y, y_test, preds, _times)}")

Random Forest C-Index : 0.9463567924767654
Random Forest Integrated Brier Score : 0.1475495122435265


# Gradient Boosting Survival Analysis

In [38]:

est = GradientBoostingSurvivalAnalysis(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
est.fit(X_train, y_train)
print(f"Gradient Boosting C-Index : {est.score(X_test, y_test)}")
survs = est.predict_survival_function(X_test)
_times = np.arange(1, 7)
preds = np.asarray([[fn(t) for t in _times] for fn in survs])
print(f"Gradient Boosting Integrated Brier Score : {integrated_brier_score(Y, y_test, preds, _times)}")

Gradient Boosting C-Index : 0.9569586816591994
Gradient Boosting Integrated Brier Score : 0.07282386448491485


Random Forest C-Index : 0.9569586816591994
Random Forest Brier Score : 0.07282386448491485
